In [10]:
from __future__ import division, print_function, absolute_import, unicode_literals
import numpy as np

import pygsti
from pygsti.extras import circuit

# Tests for the symplectic code
This is just tests, and doesn't give any info on what of this is. This should run most of the functions in symplectic.py, but currently not absolutely everything.

In [11]:
# Tests that check the symplectic random sampler, check_symplectic, the convention converter,
# and the symplectic form constructing function are working.

# Randomly have odd or even dimension (so things might behave differently for odd or even)
n = 4 + np.random.randint(0,2)

omega_S = circuit.symplectic_form(n,convention='standard')
#print(omega_S)
omega_DS = circuit.symplectic_form(n,convention='directsum')
#print(omega_DS)
omega_DStoS = circuit.change_symplectic_form_convention(omega_DS)
assert(np.array_equal(omega_S,omega_DStoS))
omega_StoDS = circuit.change_symplectic_form_convention(omega_S,outconvention='directsum')
assert(np.array_equal(omega_DS,omega_StoDS))

# Pick a random symplectic matrix in the standard convention and check it is symplectic
s_S = circuit.symplectic.random_symplectic_matrix(n)
assert(circuit.check_symplectic(s_S))

# Pick a random symplectic matrix in the directsum convention and check it is symplectic
s_DS = circuit.symplectic.random_symplectic_matrix(n,convention='directsum')
assert(circuit.check_symplectic(s_DS,convention='directsum'))

# Convert the directsum convention to the standard convention and check the output is symplectic in new convention
s_DStoS =  circuit.change_symplectic_form_convention(s_DS)
assert(circuit.check_symplectic(s_DStoS,convention='standard'))

# Convert back to the directsum convention, and check the original matrix is recovered.
s_DStoStoDS =  circuit.change_symplectic_form_convention(s_DStoS,outconvention='directsum')
assert(np.array_equal(s_DS,s_DStoStoDS))

# Check that the inversion function is working.
sin = circuit.inverse_symplectic(s_S)
assert(np.array_equal(circuit.matrixmod2.dotmod2(sin,s_S),np.identity(2*n,int)))
assert(np.array_equal(circuit.matrixmod2.dotmod2(s_S,sin),np.identity(2*n,int)))

In [12]:
# Check the Clifford sampler runs.
s, p = circuit.random_clifford(n)

# Check that a randomly sampled Clifford is a valid Clifford
assert(circuit.check_valid_clifford(s,p))

# Check the inverse Clifford function runs, and gives a valid Clifford
sin, pin = circuit.inverse_clifford(s,p)
assert(circuit.check_valid_clifford(sin,pin))

# Check the symplectic matrix part of the inverse Clifford works
assert(np.array_equal(circuit.matrixmod2.dotmod2(sin,s),np.identity(2*n,int)))
assert(np.array_equal(circuit.matrixmod2.dotmod2(s,sin),np.identity(2*n,int)))

# Check that the composite Clifford function runs, and works correctly in the special case whereby
# one Clifford is the inverse of the other.
scomp, pcomp = circuit.compose_cliffords(s,p,sin,pin)
assert(np.array_equal(scomp,np.identity(2*n,int)))
assert(np.array_equal(pcomp,np.zeros(2*n,int)))

# Check the p returned is unchanged when the seed is valid.
pvalid = circuit.construct_valid_phase_vector(s,p)
assert(np.array_equal(p,pvalid))

# Check that p returned is a valid Clifford when the input pseed is not
pseed = (p - 1) % 2
pvalid = circuit.construct_valid_phase_vector(s,pseed)
assert(circuit.check_valid_clifford(s,pvalid))

In [13]:
# Get the full hard-coded symplectic rep dictionaries
sdict, pdict = circuit.symplectic_representation()
assert('CNOT' in list(sdict.keys()))

# Get the a subset of the full hard-coded symplectic rep dictionaries
sdict, pdict = circuit.symplectic_representation(['CNOT'])
assert('CNOT' in list(sdict.keys()))

In [14]:
# Define a Clifford circuit to test `composite_clifford_from_clifford_circuit`
glist = []
glist.append(circuit.Gate('CPHASE',(0,1)))
glist.append(circuit.Gate('SWAP',(0,3)))
glist.append(circuit.Gate('H',(2)))
glist.append(circuit.Gate('P',(2)))
glist.append(circuit.Gate('HP',(0)))
glist.append(circuit.Gate('PH',(2)))
glist.append(circuit.Gate('HPH',(1)))
glist.append(circuit.Gate('CNOT',(2,0)))

c = circuit.Circuit(gate_list=glist,n=n)

# This checks the function runs when we don't provide a symplectic library.
s, p = circuit.composite_clifford_from_clifford_circuit(c)

# This checks the function runs when we do provide a symplectic library.
sdict, pdict = circuit.symplectic_representation()
s, p = circuit.composite_clifford_from_clifford_circuit(c,s_dict = sdict, p_dict = pdict)

# Define a Clifford circuit that composes to the identity
glist = []
glist.append(circuit.Gate('CPHASE',(0,1)))
glist.append(circuit.Gate('CPHASE',(1,0)))
glist.append(circuit.Gate('H',(2)))
glist.append(circuit.Gate('H',(2)))
glist.append(circuit.Gate('X',(3)))
glist.append(circuit.Gate('X',(3)))

c = circuit.Circuit(gate_list=glist,n=n)

# Check the composite clifford functions says this circuit is the identity. 
s, p = circuit.composite_clifford_from_clifford_circuit(c)
assert(np.array_equal(scomp,np.identity(2*n,int)))
assert(np.array_equal(pcomp,np.zeros(2*n,int)))

In [15]:
H = (1/np.sqrt(2))*np.array([[1.,1.],[1.,-1.]],complex)
s, p = circuit.unitary_to_symplectic_1Q(H)
assert(np.array_equal(s,sdict['H']))
assert(np.array_equal(p,pdict['H']))
s, p = circuit.unitary_to_symplectic(H)
assert(np.array_equal(s,sdict['H']))
assert(np.array_equal(p,pdict['H']))


CNOT = np.array([[1.,0.,0.,0.],[0.,1.,0.,0.],[0.,0.,0.,1.],[0.,0.,1.,0.]],complex)
s, p = circuit.unitary_to_symplectic_2Q(CNOT)
assert(np.array_equal(s,sdict['CNOT']))
assert(np.array_equal(p,pdict['CNOT']))
s, p = circuit.unitary_to_symplectic(CNOT)
assert(np.array_equal(s,sdict['CNOT']))
assert(np.array_equal(p,pdict['CNOT']))